# Functions

Exemplo de reserva de restaurantes com functions


In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("AZURE_OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"

RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

client = AzureOpenAI(
    api_version=os.getenv("AZURE_OPENAI_VERSION",""),
    azure_endpoint=RESOURCE_ENDPOINT,
    api_key = API_KEY,
)


In [2]:
def build_message(role, content):
    return {"role":role, "content":content}

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "bookRestaurant",
            "description": "Book a restaurant",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "name of the restaurant",
                    },
                    "persons": {
                        "type": "integer",
                        "description": "quantity of persons for the reservation",
                    },
                    "date": {
                        "type": "string",
                        "description": "date for the reservation",
                    },
                    "time": {
                        "type": "string",
                        "description": "time for the reservation",
                    }
                    

                },
                "required": ["name", "persons", "date", "time"],
                "additionalProperties": False,
            },
        }
    }
]

In [15]:

system_prompt = """Você é um assistente virtual que ajuda as pessoas a encontrar um restaurante e fazer uma reserva. Data atual: 2024-09-04."""

query = "Olá"

messages = [ 
                build_message("system", system_prompt),
                build_message("user", query),
    
           ]


response = client.chat.completions.create(
  model="gpt-4o",
  messages = messages,
  temperature=1,
  max_tokens=400,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None,
  tools=tools)

response_message = response.choices[0].message
print(response_message)

messages.append(build_message("assistant", response_message.content))

ChatCompletionMessage(content='Olá! Como posso ajudar você hoje? Está procurando um restaurante ou gostaria de fazer uma reserva?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [16]:
messages.append(build_message("user", "Quero um restaurante mexicano em São Paulo"))


response = client.chat.completions.create(
  model="gpt-4o",
  messages = messages,
  temperature=1,
  max_tokens=400,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None,
  tools=tools)

response_message = response.choices[0].message
print(response_message)

messages.append(build_message("assistant", response_message.content))


ChatCompletionMessage(content='Ótimo! Aqui estão alguns restaurantes mexicanos em São Paulo que você pode gostar:\n\n1. **Mé Taco**\n   - Endereço: R. Augusta, 430 - Consolação, São Paulo - SP\n   - Descrição: Conhecido por seus tacos autênticos e atmosfera descontraída.\n\n2. **La Mexicana**\n   - Endereço: Av. São Luís, 86 - República, São Paulo - SP\n   - Descrição: Um lugar tradicional com pratos típicos mexicanos.\n\n3. **Yucatán**\n   - Endereço: R. dos Três Irmãos, 158 - Vila Progredior, São Paulo - SP\n   - Descrição: Restaurante com ambiente acolhedor e diversas opções de pratos mexicanos.\n\nVocê gostaria de mais informações sobre algum desses restaurantes ou deseja fazer uma reserva?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [17]:

messages.append(build_message("user", "Quero fazer uma reserva para 2 pessoas no Hecho em México no dia 10 de setembro às 19h"))


response = client.chat.completions.create(
  model="gpt-4o",
  messages = messages,
  temperature=1,
  max_tokens=400,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None,
  tools=tools)

response_message = response.choices[0].message
print(response_message)

messages.append(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CfgRPqIpSYECYaOxR9KidGri', function=Function(arguments='{"name":"Hecho em México","persons":2,"date":"2024-09-10","time":"19:00"}', name='bookRestaurant'), type='function')])


In [18]:

messages.append(build_message("tool", "'OK'"))
messages[-1]["tool_call_id"] = messages[-2].tool_calls[0].id	

response = client.chat.completions.create(
  model="gpt-4o",
  messages = messages,
  temperature=1,
  max_tokens=400,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None,
  tools=tools)

response_message = response.choices[0].message
print(response_message)

messages.append(build_message("assistant", response_message.content))

ChatCompletionMessage(content='Sua reserva para 2 pessoas no restaurante "Hecho em México" no dia 10 de setembro às 19h foi confirmada. \n\nSe precisar de mais alguma coisa, é só me avisar! Aproveite sua refeição!', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [19]:
messages

[{'role': 'system',
  'content': 'Você é um assistente virtual que ajuda as pessoas a encontrar um restaurante e fazer uma reserva. Data atual: 2024-09-04.'},
 {'role': 'user', 'content': 'Olá'},
 {'role': 'assistant',
  'content': 'Olá! Como posso ajudar você hoje? Está procurando um restaurante ou gostaria de fazer uma reserva?'},
 {'role': 'user', 'content': 'Quero um restaurante mexicano em São Paulo'},
 {'role': 'assistant',
  'content': 'Ótimo! Aqui estão alguns restaurantes mexicanos em São Paulo que você pode gostar:\n\n1. **Mé Taco**\n   - Endereço: R. Augusta, 430 - Consolação, São Paulo - SP\n   - Descrição: Conhecido por seus tacos autênticos e atmosfera descontraída.\n\n2. **La Mexicana**\n   - Endereço: Av. São Luís, 86 - República, São Paulo - SP\n   - Descrição: Um lugar tradicional com pratos típicos mexicanos.\n\n3. **Yucatán**\n   - Endereço: R. dos Três Irmãos, 158 - Vila Progredior, São Paulo - SP\n   - Descrição: Restaurante com ambiente acolhedor e diversas opçõe